In [2]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os


In [3]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [3]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".

*******************************************************************************************
*******************************************************************************************
STARTING WITH FRAX

In [4]:
#https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet/
#frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')
frax = sg.load_api('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet')

In [5]:
fraxCollateralQuery = frax.Query.pairPerDays(
     orderBy='id'
    ,orderDirection='desc'
    ,first=1000
    #,skipValue = skipValue
    #,where = [frax.Query.fraxCollateral.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
    ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
             #,'timestamp_gte': maxBorrowFraxTimestamp
             }

)

fraxCollateral= sg.query_df([
    fraxCollateralQuery.id
    ,fraxCollateralQuery.timestamp
    ,fraxCollateralQuery.pair.address
    ,fraxCollateralQuery.pair.symbol
    ,fraxCollateralQuery.interestPerSecond #gotta convert that to APY
    ,fraxCollateralQuery.totalBorrowValue #total borrowed
    ,fraxCollateralQuery.utilization #these two give you asset available
    ,fraxCollateralQuery.pair.collateral
    ,fraxCollateralQuery.totalCollateralValue
    ,fraxCollateralQuery.totalAssetValue

    ]
)

fraxCollateral.head(1000)

""


*******************************************************************************************
*******************************************************************************************
SILO

In [7]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

#OHM pair can be found at https://app.silo.finance/silo/0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db
#GOHM pair at https://app.silo.finance/silo/0xc413dd03555f3eb29d834b482d386b2999dc2eb0 THIS THE ONE YOU WANT

In [8]:
#I don't think this is it. SHould probably look in Markets instead.
siloCollateral = silo.Query.positions(
    orderBy='id',
    orderDirection='desc',
    first=1000,
   # skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
            #,'isCollateral':True
            #,'side': 'BORROWER'
            #,'timestamp_gte': maxSiloBorrowTimestamp
            } #gOHM market
)

siloCollateralDb = sg.query_df([
    siloCollateral.id
    ,siloCollateral.account.id
    ,siloCollateral.side
    #,siloCollateral.balance weird shit here. Their documentatino says it's "not utilized"
    ,siloCollateral.assetPositions.sToken.symbol
    ,siloCollateral.assetPositions.sToken.decimals
    ,siloCollateral.assetPositions.sTokenBalance
    ,siloCollateral.assetPositions.spToken.symbol
    ,siloCollateral.assetPositions.spToken.decimals
    ,siloCollateral.assetPositions.spTokenBalance
    ,siloCollateral.assetPositions.dToken.symbol
    ,siloCollateral.assetPositions.dToken.decimals
    ,siloCollateral.assetPositions.dTokenBalance
])

siloCollateralDb.head(1000)

,positions_id,positions_account_id,positions_side,positions_assetPositions_sToken_symbol,positions_assetPositions_sToken_decimals,positions_assetPositions_sTokenBalance,positions_assetPositions_spToken_symbol,positions_assetPositions_spToken_decimals,positions_assetPositions_spTokenBalance,positions_assetPositions_dToken_symbol,positions_assetPositions_dToken_decimals,positions_assetPositions_dTokenBalance
0,0xfe9d1bd54e344dc8c4bd93e7935e39af452e5d41-0xc...,0xfe9d1bd54e344dc8c4bd93e7935e39af452e5d41,LENDER,sgOHM,18,183085805786400265,spgOHM,18,0,dgOHM,18,0
1,0xfe9d1bd54e344dc8c4bd93e7935e39af452e5d41-0xc...,0xfe9d1bd54e344dc8c4bd93e7935e39af452e5d41,BORROWER,sXAI-gOHM,18,0,spXAI-gOHM,18,0,dXAI-gOHM,18,397176069106358038436
2,0xee1520f94f304e8d551cbf310fe214212e3ca34a-0xc...,0xee1520f94f304e8d551cbf310fe214212e3ca34a,LENDER,sgOHM,18,0,spgOHM,18,0,dgOHM,18,0
3,0xe12baf0cbd6b0b173b6a5af688312f652fd57c33-0xc...,0xe12baf0cbd6b0b173b6a5af688312f652fd57c33,LENDER,sXAI-gOHM,18,0,spXAI-gOHM,18,0,dXAI-gOHM,18,0
4,0xdedcf5806c4968c6397eee97e68047bda339d0c1-0xc...,0xdedcf5806c4968c6397eee97e68047bda339d0c1,LENDER,sgOHM,18,0,spgOHM,18,0,dgOHM,18,0
5,0xdedcf5806c4968c6397eee97e68047bda339d0c1-0xc...,0xdedcf5806c4968c6397eee97e68047bda339d0c1,LENDER,sgOHM,18,17241922613955546,spgOHM,18,0,dgOHM,18,0
6,0xdedcf5806c4968c6397eee97e68047bda339d0c1-0xc...,0xdedcf5806c4968c6397eee97e68047bda339d0c1,LENDER,sXAI-gOHM,18,0,spXAI-gOHM,18,0,dXAI-gOHM,18,0
7,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880-0xc...,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,LENDER,sgOHM,18,0,spgOHM,18,0,dgOHM,18,0
8,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880-0xc...,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,BORROWER,sXAI-gOHM,18,0,spXAI-gOHM,18,0,dXAI-gOHM,18,0
9,0xcb6e1613029d790c00f89296808f278d6dc25b2f-0xc...,0xcb6e1613029d790c00f89296808f278d6dc25b2f,LENDER,sWETH-gOHM,18,1000000000000000,spWETH-gOHM,18,0,dWETH-gOHM,18,0
